## Features from Bergholz et al; 2012

1. l_numexternal = Number of external links
2. l_numdeceptive = Number of deceptive links
3. l_sus = Suspicious words in link text
4. l_numip = Links with IP addresses

## Features from Khonji et al; 2012 ##

1.   b_dear = Binaray feature that returns 1 if word "dear" is found in the body of the message , else 0
2.   b_form = Does the message contain HTML form? (binary)
3. b_numwords = Total number of words found in body (cont.)
4. b_numchr = Total number of characters found in body (cont.)
5. b_numunique = A feature that returns the
total number of unique words found in the body a given
email message
6. b_numfunct = A feature that returns
the total number of function words found in the body of
a given email



## Overall Goal:
Identify if the following features from the academic papers can help us increase accuracy in our models

In [1]:
# Import necessary libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import email
from datetime import datetime
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
import matplotlib.ticker as ticker

In [ ]:
# EDA (Exploratory Data Analysis)

print("Dataset Distribution")
class_counts =